In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.manifold import TSNE
import numpy as np
import config

Plot the overlap of source and target data in the T-SNE space.

In [ ]:
# for one run and one div step
feats = ['abs_energy','intermittency','mean','median','kurtosis','skewness','standard_deviation','agg_autocorrelation_max','erraticness','agg_linear_trend_slope']

for run in ["case1_run1"]: #, "case1_run2", "case1_run3", "case1_run4", "case1_run5"
    concat = pd.read_parquet(f"{config.path_to_data}/{run}/interim/CONCAT_feat.parquet")[['id_ts', 'data']+feats]
    for div in ["10"]: #, "5", "10" 
        target_datasets = []
        target_id_list = []
        for p in ["5000", "12000", "25000", "50000", "100000", "all"]: 
            target = pd.read_parquet(f"{config.path_to_data}/{run}/interim/target_div{div}_sim{p}{run}/target_div{div}_sim{p}{run}.parquet")
            target_ids = target.id_ts.unique().tolist()
            target_id_list.extend(target_ids)
            target_feat = concat[concat["id_ts"].isin(target_id_list)]
            target_feat["st"] = "Target"
            target_datasets.append(target_feat)
        source = pd.read_parquet(f"{config.path_to_data}/{run}/interim/source_div{div}{run}/source_div{div}{run}.parquet")
        source_feat = concat[concat.id_ts.isin(source.id_ts.unique())]
        source_feat["st"] = "Source"

In [ ]:
# draw proportional samples from the source and the targets
total_source = source_feat.id_ts.nunique()
total_combined = total_source + target_feat.id_ts.nunique()
source_ratio = total_source / total_combined 
target_ratio = 1 - source_ratio               

total_samples = 3000
source_samples_needed = int(total_samples * source_ratio)  
target_samples_needed = round((total_samples - source_samples_needed)/6) 

# calculate scaling factor on all source features
source_features_only = source_feat.drop(columns=["data", "id_ts", "st"])
scaler = StandardScaler()
source_features_scaled = pd.DataFrame(
    scaler.fit_transform(source_features_only), 
    columns=source_features_only.columns
)
source_features_scaled.dropna(axis=1, inplace=True)

# Sample source series
source_final_indices = np.random.choice(len(source_features_scaled), 
                                       min(source_samples_needed, len(source_features_scaled)), 
                                       replace=False)
source_sampled_data = source_features_scaled.iloc[source_final_indices]
source_sampled_labels = pd.Series(["Source"] * len(source_sampled_data), name="st")

print(f"Source samples: {len(source_sampled_data)}")
print(f"Target samples per iteration: ~{target_samples_needed}")

all_tsne_results = []

# scale and sample series for each target
for p in range(6):
    target_feat = target_datasets[p]
    
    # Process target data
    target_features_only = target_feat.drop(columns=["data", "id_ts", "st"])
    target_features_scaled = pd.DataFrame(
        scaler.transform(target_features_only),  # Use same scaler as source
        columns=target_features_only.columns
    )
    target_features_scaled.dropna(axis=1, inplace=True)
    
    target_available = len(target_features_scaled)
    target_sample_size = min(target_samples_needed, target_available)
    
    target_sampled_indices = np.random.choice(target_available, target_sample_size, replace=False)
    target_sampled_data = target_features_scaled.iloc[target_sampled_indices]
    target_sampled_labels = pd.Series(["Target"] * len(target_sampled_data), name="st")
    
    # Combine source and target samples
    combined_data = pd.concat([source_sampled_data, target_sampled_data], ignore_index=True)
    combined_labels = pd.concat([source_sampled_labels, target_sampled_labels], ignore_index=True)
    
    # t-SNE on the combined data
    tsne = TSNE(
        n_components=3, 
        random_state=42, 
        perplexity=min(30, len(combined_data)//3),
        init='pca',
        n_iter=300
    )
    tsne_result = tsne.fit_transform(combined_data)
    
    # Create DataFrame for plotting
    tsne_df = pd.DataFrame(tsne_result, columns=["TSNE1", "TSNE2", "TSNE3"])
    tsne_df["Data Set"] = combined_labels.reset_index(drop=True)
    
    all_tsne_results.append(tsne_df)

fig = make_subplots(
    rows=2, cols=3,
    specs=[[{'type': 'scatter3d'}]*3]*2,
    subplot_titles=[f'p = {i}' for i in ["5,000", "12,000", "25,000", "50,000", "100,000", "all"]],
    horizontal_spacing=0.05,
    vertical_spacing=0.1  
)

for annotation in fig.layout.annotations:
    annotation.font.size = 24

color_map = {'Source': 'red', 'Target': 'blue'}
symbol_map = {'Source': 'circle', 'Target': 'square'}

for idx, df in enumerate(all_tsne_results):
    row = (idx // 3) + 1
    col = (idx % 3) + 1
    
    for label in df['Data Set'].unique():
        subset = df[df['Data Set'] == label]
        fig.add_trace(
            go.Scatter3d(
                x=subset['TSNE1'],
                y=subset['TSNE2'],
                z=subset['TSNE3'],
                mode='markers',
                marker=dict(
                    size=4,
                    color=color_map[label],
                    symbol=symbol_map[label],
                    opacity=0.7
                ),
                name=label if idx == 0 else None,
                legendgroup=label,
                showlegend=(idx == 0)
            ),
            row=row, col=col
        )

fig.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white',
    width=1800,
    height=1200,
    legend=dict(
        font=dict(size=28),              
        bgcolor='rgba(255,255,255,0.9)', 
        orientation="h",
        yanchor="bottom",
        y=1.08,                          
        xanchor="right",
        x=1,
        itemwidth=100,                   
        itemsizing='constant'
    ),
    margin=dict(l=20, r=20, t=30, b=20)
)


scene_settings = dict(
    xaxis=dict(showticklabels=False, title="", showgrid=True, gridcolor='lightgray', showline=True, linewidth=1, linecolor='black'),
    yaxis=dict(showticklabels=False, title="", showgrid=True, gridcolor='lightgray', showline=True, linewidth=1, linecolor='black'),
    zaxis=dict(showticklabels=False, title="", showgrid=True, gridcolor='lightgray', showline=True, linewidth=1, linecolor='black'),
    aspectmode='cube',
    camera=dict(eye=dict(x=1.8, y=1.8, z=1.8))
)


for i in range(1, 7):
    fig.update_scenes(scene_settings, row=(i-1)//3+1, col=(i-1)%3+1)

# plotting and saving requires the packages nbformat and kaleido
#fig.write_image(f"{config.path_to_evaluation}/combined_tsne_source_div{div}.png", width=1800, height=1200, scale=2)
#fig.write_image(f"{config.path_to_evaluation}/combined_tsne_source_div{div}.pdf", width=1800, height=1200)
#fig.show()